<a href="https://colab.research.google.com/github/jpraveenkanna/Practice-notebooks/blob/master/TF_lite_Converting_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.0

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import keras


In [11]:

model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])

model.compile(optimizer='sgd', loss='mean_squared_error')

#Defining input and output
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-2.0, 1.0, 4.0, 7.0, 10.0, 13.0], dtype=float)

model.fit(xs, ys, epochs=100)


Train on 6 samples
Epoch 1/100
6/6 [==============================] - 0s 27ms/sample - loss: 82.7317
Epoch 2/100
6/6 [==============================] - 0s 287us/sample - loss: 65.0926
Epoch 3/100
6/6 [==============================] - 0s 477us/sample - loss: 51.2150
Epoch 4/100
6/6 [==============================] - 0s 464us/sample - loss: 40.2967
Epoch 5/100
6/6 [==============================] - 0s 378us/sample - loss: 31.7067
Epoch 6/100
6/6 [==============================] - 0s 312us/sample - loss: 24.9483
Epoch 7/100
6/6 [==============================] - 0s 312us/sample - loss: 19.6312
Epoch 8/100
6/6 [==============================] - 0s 324us/sample - loss: 15.4478
Epoch 9/100
6/6 [==============================] - 0s 295us/sample - loss: 12.1565
Epoch 10/100
6/6 [==============================] - 0s 337us/sample - loss: 9.5670
Epoch 11/100
6/6 [==============================] - 0s 309us/sample - loss: 7.5296
Epoch 12/100
6/6 [==============================] - 0s 370us/sample -

In [2]:
print(model.predict([3.0]))

[[9.998353]]


###Saving the Tensorflow model 

In [6]:
model.save("Single_Layer_Model",save_format=tf)

INFO:tensorflow:Assets written to: Single_Layer_Model/assets


In [4]:
saved_model_dir = '/content/Single_Layer'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

780

In [17]:
ls

converted_model.tflite       sample_data/
deeplabv3_pascal_train_aug/  Single_Layer_Model/


###Using tflite_convert CLI to convert pretained model

In [0]:
!curl 'https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_0.50_128_frozen.tgz' \
  | tar xzv -C /tmp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0mobilenet_v1_0.50_128/
mobilenet_v1_0.50_128/quantized_graph.pb
mobilenet_v1_0.50_128/labels.txt
mobilenet_v1_0.50_128/frozen_graph.pb
100 6143k  100 6143k    0     0  8180k      0 --:--:-- --:--:-- --:--:-- 8169k


In [0]:
!tflite_convert \
  --output_file=/tmp/converted_model.tflite \
  --graph_def_file=/tmp/mobilenet_v1_0.50_128/frozen_graph.pb \
  --input_arrays=input \
  --output_arrays=MobilenetV1/Predictions/Reshape_1

2019-12-22 09:14:50.465020: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-22 09:14:50.529467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-12-22 09:14:50.530065: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2019-12-22 09:14:50.542879: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2019-12-22 09:14:50.816265: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2019-12-22 09:14:50.951935: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuf

In [0]:
#deeplab pretrained model
#http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz

In [7]:
!curl 'http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz' \
  | tar xzv -C /content

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0deeplabv3_pascal_train_aug/
deeplabv3_pascal_train_aug/model.ckpt.index
deeplabv3_pascal_train_aug/frozen_inference_graph.pb
 25  438M   25  109M    0     0  65.5M      0  0:00:06  0:00:01  0:00:05 65.5Mdeeplabv3_pascal_train_aug/model.ckpt.data-00000-of-00001
100  438M  100  438M    0     0  58.8M      0  0:00:07  0:00:07 --:--:-- 55.8M


In [16]:
!tflite_convert \
  --output_file=/content/foo.tflite \
  --graph_def_file=/content/deeplabv3_pascal_train_aug \
  --input_arrays=input \
  --output_arrays=deeplabv3/Predictions/Reshape_1 \
  --saved_model_dir=/content/deeplabv3_pascal_train_aug \

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I1222 10:45:51.442835 140113500379008 saver.py:1503] Saver not created because there are no variables in the graph to restore
2019-12-22 10:45:51.445007: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-12-22 10:45:51.449049: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-12-22 10:45:51.449891: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1a25a8c0 executing computations on platform Host. Devices:
2019-12-22 10:45:51.449927: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Traceback (most recent call last):
  File "/usr/local/bin/tflite_convert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/lite/python/tflite_convert.py", line 